In [1]:
import pandas as pd

In [ ]:
#Código Cidade SP (IBGE) = 3550308

In [ ]:
#Número de funcionários e infraestrutura (Cozinha e refeitório) 

In [ ]:
#Explicação das colunas selecionadas
#IN_COZINHA = Dependências existentes na escola - Cozinha (0-Não,1-Sim)
#IN_REFEITORIO = Dependências existentes na escola - Refeitório (0-Não,1-Sim)
#IN_ALIMENTACAO = Alimentação escolar para os alunos (0-Não,1-Sim)
#NU_FUNCIONARIOS = Total de funcionários da escola (inclusive profissionais escolares em sala de aula)

In [ ]:
df = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2015\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                usecols=['CO_ENTIDADE','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO','IN_COZINHA',
                        'IN_REFEITORIO','IN_ALIMENTACAO','NU_FUNCIONARIOS'])

In [ ]:
#Selecionando as escolas com Código de Município = 3550308 (Cidade de SP), TP_DEPENDENCIA = 2 (Escolas Estaduais) e TP_SITUACAO_FUNCIONAMENTO (Escolas 'Em Atividade')
df_sp = df[(df['CO_MUNICIPIO'] == 3550308)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]

In [ ]:
#Número de alunos

In [ ]:
chunk = pd.read_csv(r'C:\Users\guilh\Documents\CENSO_ESCOLAR\2018\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_MUNICIPIO','CO_ENTIDADE','TP_DEPENDENCIA'],chunksize = 100000)

In [ ]:
#Selecionando as escolas com Código de Município = 3550308 (Cidade de SP) e TP_DEPENDENCIA = 2 (Escolas Estaduais)
ls = []
for data in chunk:
    data = data[(data['CO_MUNICIPIO'] == 3550308)&(data['TP_DEPENDENCIA'] == 2)]
    ls.append(data)
df_estud = pd.concat(ls)

In [ ]:
#Retirando duplicatas na coluna ID dos matriculados 
df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])

In [ ]:
#Agrupando por código de escola e contando 
df_estud = df_estud.groupby('CO_ENTIDADE').count().reset_index()[['CO_ENTIDADE','CO_PESSOA_FISICA']]

In [ ]:
pd.merge(df_sp,df_estud,on=['CO_ENTIDADE'])

In [ ]:
#Operacionalização do processo

In [2]:
writer = pd.ExcelWriter('info_censo_escolar_cidade_sp.xlsx', engine='xlsxwriter')

In [3]:
for i in range(2015,2020):
    if (i==2015)|(i==2016)|(i==2017):
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO','IN_COZINHA','IN_REFEITORIO','IN_ALIMENTACAO','NU_FUNCIONARIOS'])
        df_sp = df[(df['CO_MUNICIPIO'] == 3550308)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['CO_PESSOA_FISICA','NU_IDADE','CO_MUNICIPIO','CO_ENTIDADE','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_MUNICIPIO'] == 3550308)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['CO_PESSOA_FISICA'])
        df_estud = df_estud.groupby('CO_ENTIDADE').count().reset_index()[['CO_ENTIDADE','CO_PESSOA_FISICA']]
        df = pd.merge(df_sp,df_estud,on=['CO_ENTIDADE'])
        df.sort_values(by=['CO_ENTIDADE'],inplace=True)
        print(i)
        df.to_excel(writer, index = False, sheet_name=f'Tabela_{i}')
    elif (i==2018):
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,
                    usecols=['CO_ENTIDADE','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO','IN_COZINHA','IN_REFEITORIO','IN_ALIMENTACAO','QT_FUNCIONARIOS'])
        df_sp = df[(df['CO_MUNICIPIO'] == 3550308)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['ID_ALUNO','NU_IDADE','CO_MUNICIPIO','CO_ENTIDADE','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_MUNICIPIO'] == 3550308)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['ID_ALUNO'])
        df_estud = df_estud.groupby('CO_ENTIDADE').count().reset_index()[['CO_ENTIDADE','ID_ALUNO']]
        df = pd.merge(df_sp,df_estud,on=['CO_ENTIDADE'])
        df.sort_values(by=['CO_ENTIDADE'],inplace=True)
        print(i)
        df.to_excel(writer, index = False, sheet_name=f'Tabela_{i}')
    else:
        df = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\ESCOLAS.csv',sep='|',encoding='Latin1',low_memory=False,)
        df['QT_FUNCIONARIOS'] = df.iloc[:,150:163].sum(axis=1)
        df = df[['CO_ENTIDADE','CO_MUNICIPIO','TP_DEPENDENCIA','TP_SITUACAO_FUNCIONAMENTO','IN_COZINHA','IN_REFEITORIO','IN_ALIMENTACAO','QT_FUNCIONARIOS']]
        df_sp = df[(df['CO_MUNICIPIO'] == 3550308)&(df['TP_DEPENDENCIA'] == 2)&(df['TP_SITUACAO_FUNCIONAMENTO'] == 1)]
        chunk = pd.read_csv(rf'C:\Users\guilh\Documents\CENSO_ESCOLAR\{i}\DADOS\MATRICULA_SUDESTE.csv',sep='|',encoding='Latin1',low_memory=False,
                        usecols=['ID_ALUNO','NU_IDADE','CO_MUNICIPIO','CO_ENTIDADE','TP_DEPENDENCIA'],chunksize = 100000)
        ls = []
        for data in chunk:
            data = data[(data['CO_MUNICIPIO'] == 3550308)&(data['TP_DEPENDENCIA'] == 2)]
            ls.append(data)
        df_estud = pd.concat(ls)
        df_estud = df_estud.drop_duplicates(subset=['ID_ALUNO'])
        df_estud = df_estud.groupby('CO_ENTIDADE').count().reset_index()[['CO_ENTIDADE','ID_ALUNO']]
        df = pd.merge(df_sp,df_estud,on=['CO_ENTIDADE'])
        df.sort_values(by=['CO_ENTIDADE'],inplace=True)
        print(i)
        df.to_excel(writer, index = False, sheet_name=f'Tabela_{i}')

2015
2016
2017
2018
2019


In [4]:
writer.save()